In [47]:
import pandas as pd
import os
import fitz #para número de páginas pdf
import tabula #para obtener datos del pd
import subprocess
from datetime import datetime
import numpy as np

In [48]:
input = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\01_Input'
output = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\02_Output'
temp_path = r'C:\David\Desarrollos\02_Procesos\09_Control_de_Saldos\02_Python\07_Temp'

In [49]:
archivos_xls = [file.name for file in os.scandir(input) if file.name.endswith('.xls')]
#archivos_xlsx = [file.name for file in os.scandir(input) if file.name.endswith('.xlsx')]
archivos_pdf = [file.name for file in os.scandir(input) if file.name.endswith('.pdf')]

In [50]:
tasa_diaria_usd = ((1 + 0.04) ** (1 / 360)) - 1
tasa_diaria_pen = ((1 + 0.0425) ** (1 / 360)) - 1

In [51]:
def obtener_numero_paginas_pdf(archivo):
    pdf_document = fitz.open(archivo)
    return pdf_document.page_count

In [52]:
def tratar_df_brutos_de_pdf(max_proc, contable_null, df):
    #print(max_proc, contable_null)

    if max_proc == 5 and not contable_null:
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc == 5 and contable_null:
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE']]
        df_columnas_por_ubicacion = df.iloc[:, [1, -2]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and not contable_null:
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)
    
    if max_proc != 5 and contable_null:
        #v Inicio de parte no testeada por caso no presentado (se incluye porque el caso en el que max_prox == 5 y contable_null es True sí se ha dado)
        df = df.drop(columns = ['CONTABLE'])
        df = df.rename(columns = {df.columns[-1]: 'CONTABLE'})     
        #^ Fin de parte no testeada por caso no presentado        
        df['Columna 2'] = df['PROC.'].str[5:]
        df['PROC.'] = df['PROC.'].str[:5]
        df_columnas_con_nombres = df[['PROC.', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CONTABLE', 'Columna 2']]
        df_columnas_por_ubicacion = df.iloc[:, [-3]]
        return pd.concat([df_columnas_con_nombres, df_columnas_por_ubicacion], axis = 1)

In [53]:
def generar_dataframe_desde_pagina_pdf(archivo, pagina):
    if pagina == 1:
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (350, 0, 842, 595))[0]
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})        
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]
    
    if pagina > 1:
        df = tabula.read_pdf(archivo, pages=f'{pagina}', stream=True, encoding='ISO-8859-1', area = (195, 0, 842, 595))[0]
        df = df[~(df['PROC.'].isna())] #Lee siempre la primera línea (no encabezados) como nulo
        max_proc = df['PROC.'].str.len().max()
        contable_null = (pd.isna(df['CONTABLE'].unique())[0] == True)
        df = tratar_df_brutos_de_pdf(max_proc, contable_null, df)
        df = df.rename(columns = {df.columns[-2]: 'DESCRIPCIÓN', df.columns[-1]: 'CARGO_ABONO', 'PROC.': 'FECHA', 'CONTABLE': 'SALDO CONTABLE'})  
        return df[['FECHA', 'DESCRIPCIÓN', 'SUC-AGE', 'NUM OP', 'HORA', 'ORIGEN', 'TIPO', 'CARGO_ABONO', 'SALDO CONTABLE']]

In [54]:
def corregir_fecha_pdf(fila, mes):
    fecha_operacion = fila['FECHA']
    year = str(fila['YEAR'])
    return datetime.strptime(fecha_operacion + "-" + year, '%d-%m-%Y')

In [55]:
def generar_dataframe_desde_pdf(archivo, empresa, moneda, banco):
    ruta_archivo = os.path.join(input, archivo)
    numero_paginas_pdf = obtener_numero_paginas_pdf(ruta_archivo)
    dfs_temp = []
    #Es numero_paginas_pdf - 1 porque la última página de los pdf siempre muestra información no relevante
    for pagina in range(numero_paginas_pdf - 1):
        #Es pagina + 1 porque el pdf empieza desde página 1 y el range empieza desde 0
        if pagina == 0:
            df_temp_1 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_1)
        else:
            df_temp_2 = generar_dataframe_desde_pagina_pdf(ruta_archivo, pagina + 1)
            dfs_temp.append(df_temp_2)
    df = pd.concat(dfs_temp)
    #Agrega columna año
    df['YEAR'] = archivo[8:12]
    mes = str(df[~(df['FECHA'].isna())]['FECHA'].str[3:5].unique()[0])
    df['FECHA'] = df.apply(corregir_fecha_pdf, axis = 1, args = [mes,])
    #Corrigiendo tipo de dato de columnas float
    df['CARGO_ABONO'] = df['CARGO_ABONO'].apply(lambda x: '-0' + str(x)[:-1] if not pd.isna(x) and str(x).endswith('-') else x) #maneja los signos negativos que están al final de la cadena
    df['CARGO_ABONO'] = df['CARGO_ABONO'].str.replace(',', '').astype(float)
    df['SALDO CONTABLE'] = df['SALDO CONTABLE'].apply(lambda x: float(str(x).replace(',', '')) if not pd.isna(x) else x)

    #Desechando columna año auxiliar
    df = df.drop(columns = ['YEAR'])
    df = df[['FECHA', 'DESCRIPCIÓN', 'CARGO_ABONO', 'SALDO CONTABLE']].rename(columns = {'SALDO CONTABLE': 'SALDO_CONTABLE', 'DESCRIPCIÓN': 'DESCRIPCION'})

    #Rellenando fechas que no existen en extractos bancario
    min_date = df['FECHA'].dt.date.min()
    min_date = pd.Timestamp(year = min_date.year, month = min_date.month, day = 1)
    #max_date = df['FECHA'].dt.date.max()
    max_date = min_date + pd.offsets.MonthEnd(0)

    full_dates = pd.date_range(start = min_date, end = max_date)
    df_full_dates = pd.DataFrame(columns = ['FECHA'], data = full_dates)
    
    df_full_dates = df_full_dates.merge(df, on = 'FECHA', how = 'left')
    
    #Agregando columnas de banco, moneda y empresa
    df['EMPRESA'] = empresa
    df['MONEDA'] = moneda
    df['BANCO'] = banco
    df_full_dates['EMPRESA'] = empresa
    df_full_dates['MONEDA'] = moneda
    df_full_dates['BANCO'] = banco

    df_full_dates = df_full_dates[['FECHA', 'SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']] 
    
    return df, df_full_dates

In [56]:
def corregir_xls(archivo):
    #Script de Power Shell
    powershell_script = f"""
    $OriginalFilePath = "{input + '\\' + archivo}"
    $NewFilePath = "{temp_path + '\\' + archivo + 'x'}"
    $Excel = New-Object -ComObject Excel.Application
    $Excel.Visible = $false
    $Workbook = $Excel.Workbooks.Open($OriginalFilePath)
    $Workbook.SaveAs($NewFilePath, 51)
    $Workbook.Close()
    $Excel.Quit()
    [System.Runtime.Interopservices.Marshal]::ReleaseComObject($Excel) | Out-Null
    [System.Runtime.Interopservices.Marshal]::ReleaseComObject($Workbook) | Out-Null
    """
    subprocess.run(["powershell", "-Command", powershell_script])
    return archivo + 'x'

In [57]:
def corregir_fecha_xls(fila, mes):

    fecha_operacion = fila['FECHA OPER.']
    year = str(fila['YEAR'])

    if not pd.isna(fecha_operacion):
        return datetime.strptime(fecha_operacion[:5] + "-" + year, '%d-%m-%Y')
    else:
        return datetime.strptime('01' + "-" + mes + "-" + year, '%d-%m-%Y')

In [58]:
def generar_dataframe_desde_xls(archivo, empresa, moneda, banco):
    
    if str(archivo).endswith('.xls'):
        nuevo_archivo = corregir_xls(archivo)

    df = pd.read_excel(os.path.join(temp_path, nuevo_archivo))
    primera_columna = df.columns[0]
    skip_rows_dinamico = int(df[df[primera_columna].fillna('blank').str.contains('FECHA OPER.')].head(1)[primera_columna].index[0]) + 1

    df = pd.read_excel(os.path.join(temp_path, nuevo_archivo), skiprows = skip_rows_dinamico)
    #Excluir registros no relacionados con transacciones
    df = df[(df['FECHA OPER.'].str.len() == 6) | (df['DESCRIPCIÓN'].str.contains('SALDO ANTERIOR'))]
    #Agrega columna año auxiliar
    df['YEAR'] = archivo[8:12]
    #Uniformizando fecha de operación. Además, asigna fecha de inicio de mes de dataframe a casos de "saldo anterior"
    mes = str(df[~(df['FECHA OPER.'].isna())]['FECHA OPER.'].str[3:5].unique()[0])
    df['FECHA OPER.'] = df.apply(corregir_fecha_xls, axis = 1, args = [mes,])
    #Corrigiendo tipo de dato de columnas float
    df['CARGO/ABONO'] = df['CARGO/ABONO'].str[:-1].str.replace(',', '').astype(float)
    df['ITF'] = df['ITF'].str[:-1].str.replace(',', '').astype(float)
    df['SALDO CONTABLE'] = df['SALDO CONTABLE'].str[:-1].str.replace(',', '').astype(float)
    
    #Desechando columna año auxiliar
    df = df.drop(columns = ['YEAR'])
    df = df[['FECHA OPER.', 'DESCRIPCIÓN', 'CARGO/ABONO', 'SALDO CONTABLE']].rename(columns = {'DESCRIPCIÓN': 'DESCRIPCION', 'FECHA OPER.': 'FECHA', 'CARGO/ABONO': 'CARGO_ABONO', 'SALDO CONTABLE': 'SALDO_CONTABLE'})

    #Rellenando fechas que no existen en extractos bancario
    min_date = df['FECHA'].dt.date.min()
    min_date = pd.Timestamp(year = min_date.year, month = min_date.month, day = 1)
    #max_date = df['FECHA'].dt.date.max()
    max_date = min_date + pd.offsets.MonthEnd(0)

    full_dates = pd.date_range(start = min_date, end = max_date)
    df_full_dates = pd.DataFrame(columns = ['FECHA'], data = full_dates)

    df_full_dates = df_full_dates.merge(df, on = 'FECHA', how = 'left')

    #Agregando columnas de banco, moneda y empresa
    df['EMPRESA'] = empresa
    df['MONEDA'] = moneda
    df['BANCO'] = banco
    df_full_dates['EMPRESA'] = empresa
    df_full_dates['MONEDA'] = moneda
    df_full_dates['BANCO'] = banco

    df_full_dates = df_full_dates[['FECHA', 'SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']] 

    return df, df_full_dates

In [59]:
try:
    #Listas para dataframes con data de bancos
    df_BCP_BPO_PEN_lista = []
    df_BCP_BPO_USD_lista = []
    df_BCP_SDP_USD_lista = []
    df_BCP_SDP_PEN_lista = []
    df_BCP_TRI_PEN_lista = []
    df_BCP_TRI_USD_lista = []

    #Listas para dataframes con data full calendar
    df_BCP_BPO_PEN_full_calendar_lista = []
    df_BCP_BPO_USD_full_calendar_lista = []
    df_BCP_SDP_USD_full_calendar_lista = []
    df_BCP_SDP_PEN_full_calendar_lista = []
    df_BCP_TRI_PEN_full_calendar_lista = []
    df_BCP_TRI_USD_full_calendar_lista = []

    for archivo in archivos_pdf:
        empresa_moneda = archivo[-11:-4]

        match empresa_moneda:
            case 'BPO PEN':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'BPO', 'PEN', 'BCP')
                df_BCP_BPO_PEN_lista.append(df_data_bancos_temp)
                df_BCP_BPO_PEN_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'BPO USD':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'BPO', 'USD', 'BCP')
                df_BCP_BPO_USD_lista.append(df_data_bancos_temp)
                df_BCP_BPO_USD_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'SDP USD':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'SDP', 'USD', 'BCP')
                df_BCP_SDP_USD_lista.append(df_data_bancos_temp)
                df_BCP_SDP_USD_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'SDP PEN':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'SDP', 'PEN', 'BCP')
                df_BCP_SDP_PEN_lista.append(df_data_bancos_temp)
                df_BCP_SDP_PEN_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'TRI PEN':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'TRI', 'PEN', 'BCP')
                df_BCP_TRI_PEN_lista.append(df_data_bancos_temp)
                df_BCP_TRI_PEN_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'TRI USD':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_pdf(archivo, 'TRI', 'USD', 'BCP')
                df_BCP_TRI_USD_lista.append(df_data_bancos_temp)
                df_BCP_TRI_USD_full_calendar_lista.append(df_data_full_calendar_temp)
            case _:
                print('Se ha detectado un caso desconocido de EMPRESA_MONEDA en los archivos pdf')

    df_BCP_BPO_PEN = pd.concat(df_BCP_BPO_PEN_lista)
    df_BCP_BPO_USD = pd.concat(df_BCP_BPO_USD_lista)
    df_BCP_SDP_USD = pd.concat(df_BCP_SDP_USD_lista)
    df_BCP_SDP_PEN = pd.concat(df_BCP_SDP_PEN_lista)
    df_BCP_TRI_PEN = pd.concat(df_BCP_TRI_PEN_lista)
    df_BCP_TRI_USD = pd.concat(df_BCP_TRI_USD_lista)

    df_BCP_BPO_PEN_full_calendar = pd.concat(df_BCP_BPO_PEN_full_calendar_lista)
    df_BCP_BPO_USD_full_calendar = pd.concat(df_BCP_BPO_USD_full_calendar_lista)
    df_BCP_SDP_USD_full_calendar = pd.concat(df_BCP_SDP_USD_full_calendar_lista)
    df_BCP_SDP_PEN_full_calendar = pd.concat(df_BCP_SDP_PEN_full_calendar_lista)
    df_BCP_TRI_PEN_full_calendar = pd.concat(df_BCP_TRI_PEN_full_calendar_lista)
    df_BCP_TRI_USD_full_calendar = pd.concat(df_BCP_TRI_USD_full_calendar_lista)
except Exception as e:
    print(e)

In [60]:
try:
    #Listas para dataframes con data de bancos
    df_BBVA_SDP_PEN_lista = []
    df_BBVA_SDP_USD_lista = []
    df_BBVA_TRI_PEN_lista = []
    df_BBVA_TRI_USD_lista = []

    #Listas para dataframes con data full calendar
    df_BBVA_SDP_PEN_full_calendar_lista = []
    df_BBVA_SDP_USD_full_calendar_lista = []
    df_BBVA_TRI_PEN_full_calendar_lista = []
    df_BBVA_TRI_USD_full_calendar_lista = []

    for archivo in archivos_xls:
        empresa_moneda = archivo[-11:-4]
        match empresa_moneda:
            case 'SDP PEN':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_xls(archivo, 'SDP', 'PEN', 'BBVA')
                df_BBVA_SDP_PEN_lista.append(df_data_bancos_temp)
                df_BBVA_SDP_PEN_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'SDP USD':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_xls(archivo, 'SDP', 'USD', 'BBVA')
                df_BBVA_SDP_USD_lista.append(df_data_bancos_temp)
                df_BBVA_SDP_USD_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'TRI PEN':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_xls(archivo, 'TRI', 'PEN', 'BBVA')
                df_BBVA_TRI_PEN_lista.append(df_data_bancos_temp)
                df_BBVA_TRI_PEN_full_calendar_lista.append(df_data_full_calendar_temp)
            case 'TRI USD':
                df_data_bancos_temp, df_data_full_calendar_temp = generar_dataframe_desde_xls(archivo, 'TRI', 'USD', 'BBVA')
                df_BBVA_TRI_USD_lista.append(df_data_bancos_temp)
                df_BBVA_TRI_USD_full_calendar_lista.append(df_data_full_calendar_temp)
            case _:
                print('Se ha detectado un caso desconocido de EMPRESA_MONEDA en los archivos xls')

    try:
        df_BBVA_SDP_PEN = pd.concat(df_BBVA_SDP_PEN_lista)
    except Exception as e:
        print(e)
    
    try:
        df_BBVA_SDP_USD = pd.concat(df_BBVA_SDP_USD_lista)
    except Exception as e:
        print(e)

    try:
        df_BBVA_SDP_PEN_full_calendar = pd.concat(df_BBVA_SDP_PEN_full_calendar_lista)
    except Exception as e:
        print(e)

    try:
        df_BBVA_SDP_USD_full_calendar = pd.concat(df_BBVA_SDP_USD_full_calendar_lista)
    except Exception as e:
        print(e)

######################################################################################################################################################
######################################################################################################################################################
######################################################################################################################################################

    try:
        df_BBVA_TRI_PEN = pd.concat(df_BBVA_TRI_PEN_lista)
    except Exception as e:
        print(e)
    
    try:
        df_BBVA_TRI_USD = pd.concat(df_BBVA_TRI_USD_lista)
    except Exception as e:
        print(e)    

    try:
        df_BBVA_TRI_PEN_full_calendar = pd.concat(df_BBVA_TRI_PEN_full_calendar_lista)
    except Exception as e:
        print(e)

    try:
        df_BBVA_TRI_USD_full_calendar = pd.concat(df_BBVA_TRI_USD_full_calendar_lista)
    except Exception as e:
        print(e)


except Exception as e:
    print(e)

In [61]:
try:
    df_BCP_BPO_PEN.to_excel(os.path.join(output, 'BCP_BPO_PEN.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BCP_BPO_USD.to_excel(os.path.join(output, 'BCP_BPO_USD.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BCP_SDP_USD.to_excel(os.path.join(output, 'BCP_SDP_USD.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BCP_SDP_PEN.to_excel(os.path.join(output, 'BCP_SDP_PEN.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BBVA_SDP_PEN.to_excel(os.path.join(output, 'BBVA_SDP_PEN.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BBVA_SDP_USD.to_excel(os.path.join(output, 'BBVA_SDP_USD.xlsx'), index = False)
except Exception as e:
    print(e)

########################################################################################################################################################
########################################################################################################################################################

try:
    df_BBVA_TRI_PEN.to_excel(os.path.join(output, 'BBVA_TRI_PEN.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BBVA_TRI_USD.to_excel(os.path.join(output, 'BBVA_TRI_USD.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BCP_TRI_PEN.to_excel(os.path.join(output, 'BCP_TRI_PEN.xlsx'), index = False)
except Exception as e:
    print(e)

try:
    df_BCP_TRI_USD.to_excel(os.path.join(output, 'BCP_TRI_USD.xlsx'), index = False)
except Exception as e:
    print(e)

In [62]:
"""#Estable sin TRI
df_Base_Saldo_Contable = pd.concat([df_BCP_BPO_PEN_full_calendar, df_BCP_BPO_USD_full_calendar, df_BCP_SDP_USD_full_calendar, df_BCP_SDP_PEN_full_calendar, df_BBVA_SDP_PEN_full_calendar, df_BBVA_SDP_USD_full_calendar])
df_Base_Saldo_Contable = df_Base_Saldo_Contable.drop_duplicates(subset = ['FECHA', 'EMPRESA', 'MONEDA', 'BANCO'], keep = 'last')
df_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']] = df_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']].ffill()
"""

"#Estable sin TRI\ndf_Base_Saldo_Contable = pd.concat([df_BCP_BPO_PEN_full_calendar, df_BCP_BPO_USD_full_calendar, df_BCP_SDP_USD_full_calendar, df_BCP_SDP_PEN_full_calendar, df_BBVA_SDP_PEN_full_calendar, df_BBVA_SDP_USD_full_calendar])\ndf_Base_Saldo_Contable = df_Base_Saldo_Contable.drop_duplicates(subset = ['FECHA', 'EMPRESA', 'MONEDA', 'BANCO'], keep = 'last')\ndf_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']] = df_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']].ffill()\n"

In [63]:
df_Base_Saldo_Contable = pd.concat([df_BCP_BPO_PEN_full_calendar, df_BCP_BPO_USD_full_calendar, df_BCP_SDP_USD_full_calendar, df_BCP_SDP_PEN_full_calendar, df_BBVA_SDP_PEN_full_calendar, df_BBVA_SDP_USD_full_calendar, \
                                    df_BBVA_TRI_PEN_full_calendar, df_BBVA_TRI_USD_full_calendar, df_BCP_TRI_PEN_full_calendar, df_BCP_TRI_USD_full_calendar])
df_Base_Saldo_Contable = df_Base_Saldo_Contable.drop_duplicates(subset = ['FECHA', 'EMPRESA', 'MONEDA', 'BANCO'], keep = 'last')
df_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']] = df_Base_Saldo_Contable[['SALDO_CONTABLE', 'EMPRESA', 'MONEDA', 'BANCO']].ffill()

In [64]:
#Agregando columna RNG
condiciones = [(df_Base_Saldo_Contable['SALDO_CONTABLE'] > 50000) & (df_Base_Saldo_Contable['MONEDA'] == 'USD'),
               (df_Base_Saldo_Contable['SALDO_CONTABLE'] > 50000) & (df_Base_Saldo_Contable['MONEDA'] == 'PEN')]
valores = [(df_Base_Saldo_Contable['SALDO_CONTABLE'] - 50000) * tasa_diaria_usd,
           (df_Base_Saldo_Contable['SALDO_CONTABLE'] - 50000) * tasa_diaria_pen]
df_Base_Saldo_Contable['RNG'] = np.select(condiciones, valores, default = 0)

In [65]:
pivot_table_df = df_Base_Saldo_Contable.pivot_table(
    index=['FECHA', 'EMPRESA', 'MONEDA'],  # Índices
    columns='BANCO',            # Columnas
    values='SALDO_CONTABLE',    # Valores
    aggfunc='sum',              # Función de agregación
    fill_value=0                # Opcional: Rellenar NaN con 0
)
pivot_table_df = pivot_table_df.reset_index()
pivot_table_df

BANCO,FECHA,EMPRESA,MONEDA,BBVA,BCP
0,2024-01-01,TRI,PEN,3569.11,12840.12
1,2024-01-01,TRI,USD,812407.45,27520.55
2,2024-01-02,TRI,PEN,3569.11,12840.12
3,2024-01-02,TRI,USD,812407.45,27520.55
4,2024-01-03,TRI,PEN,3569.11,12840.12
...,...,...,...,...,...
1093,2024-11-29,TRI,USD,0.00,22988.89
1094,2024-11-30,SDP,PEN,39503.92,0.00
1095,2024-11-30,SDP,USD,257533.06,0.00
1096,2024-11-30,TRI,PEN,0.00,259.96


In [66]:
#Agregando columna RNG
condiciones = [((pivot_table_df['BBVA'] > 50000) & (pivot_table_df['BCP'] > 50000)) & (pivot_table_df['MONEDA'] == 'USD'),
                (pivot_table_df['BBVA'] > 50000) & (pivot_table_df['MONEDA'] == 'USD'),
                (pivot_table_df['BCP'] > 50000) & (pivot_table_df['MONEDA'] == 'USD'),
                ((pivot_table_df['BBVA'] > 50000) & (pivot_table_df['BCP'] > 50000)) & (pivot_table_df['MONEDA'] == 'PEN'),
                (pivot_table_df['BBVA'] > 50000) & (pivot_table_df['MONEDA'] == 'PEN'),
                (pivot_table_df['BCP'] > 50000) & (pivot_table_df['MONEDA'] == 'PEN')]

In [67]:

valores = [((pivot_table_df['BBVA'] - 50000) + (pivot_table_df['BCP'] - 50000)) * tasa_diaria_usd,
           ((pivot_table_df['BBVA'] - 50000)) * tasa_diaria_usd,
           ((pivot_table_df['BCP'] - 50000)) * tasa_diaria_usd,
           ((pivot_table_df['BBVA'] - 50000) + (pivot_table_df['BCP'] - 50000)) * tasa_diaria_pen,
           ((pivot_table_df['BBVA'] - 50000)) * tasa_diaria_pen,
           ((pivot_table_df['BCP'] - 50000)) * tasa_diaria_pen]
pivot_table_df['RNG'] = np.select(condiciones, valores, default = 0)

In [68]:
pivot_table_df.to_excel(os.path.join(output, 'BASE_Saldo_Contable_pivot.xlsx'), index = False)

In [69]:
df_Base_Saldo_Contable.to_excel(os.path.join(output, 'BASE_Saldo_Contable.xlsx'), index = False)